## Initialization of the notebook with installation and import of correct versions of libraries.

In [1]:
!pip install torchinfo


In [2]:
from google.colab import drive
drive.mount('/content/drive', force_remount = True)

Mounted at /content/drive


In [3]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Wed May 14 09:35:37 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA L4                      Off |   00000000:00:03.0 Off |                    0 |
| N/A   46C    P8             12W /   72W |       0MiB /  23034MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [4]:
import numpy as np
import os
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
import random
import seaborn as sns
import pandas as pd
import gc
import cv2

In [5]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import torch.optim as optim

In [6]:
read_small = True

In [7]:
if read_small:
  !unzip drive/MyDrive/TecnosensSondaOttica/DatasetTS_MA_small.zip -d ./dataset
else:
  !unzip drive/MyDrive/TecnosensSondaOttica/DatasetTS_MA_FT_medium.zip -d ./dataset

unzip:  cannot find or open drive/MyDrive/TecnosensSondaOttica/DatasetTS_MA_small.zip, drive/MyDrive/TecnosensSondaOttica/DatasetTS_MA_small.zip.zip or drive/MyDrive/TecnosensSondaOttica/DatasetTS_MA_small.zip.ZIP.


In [8]:
datasetPath = "drive/MyDrive/HPPS_Nico/HPPS/Project/ModelRegression/Dataset"

In [9]:
%ls

drive/  sample_data/


## Setting a seed on all frameworks for reproducibility

In [10]:
# Random seed for reproducibility
seed = 1234

random.seed(seed)
os.environ['PYTHONHASHSEED'] = str(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)

# Load data and preprocessing

## Generate dataset

In [ ]:
labels_df = pd.read_csv(datasetPath+"/labels_df2.csv")
#if not read_small:
  #labels_df = pd.read_csv("../drive/MyDrive/TecnosensSondaOttica/labels_df_ft.csv", index_col=False)

In [ ]:
X_train = []
Y_train = []
X_val = []
Y_val = []
X_test = []
Y_test = []

#dir = "DatasetTS_MA_FT_medium"
if read_small:
  dir = datasetPath+"/DatasetTS_MA_small"
directories = os.listdir(dir)
if ".DS_Store" in directories:
    directories.remove(".DS_Store")
directories = [int(dir) for dir in directories]
directories.sort()
directories

[-5, -4, -3, -2, -1, 0, 1, 2, 3, 4, 5]

In [ ]:
np.random.seed(42)
print(gc.collect())

used_angles = [-5,-4,-3,-2,-1,0,1,2,3,4,5]
if not read_small:
  used_angles = [-5, 5]

for directory in tqdm(directories):
  if directory in used_angles:
    files = os.listdir(dir + '/' + str(directory))
    files.sort()
    files = list(reversed(files))[:1100]
    for file in tqdm(files):
      try:
        distance_str = file.split('-')[0]
        distance_val = int(distance_str)
      except:
        print(f"Impossibile estrarre distanza da {file}")
        continue

      match = labels_df[(labels_df['distance'] == distance_val) & (labels_df['angle'] == directory)]
      if len(match) == 0:
        print(f"Nessuna label trovata per file {file} con distanza {distance_val} e angolo {directory}")
        continue

      train_val_test = np.random.randint(0,100)  # 80-5-15 split

      image = cv2.imread(dir + "/" + str(directory) + "/" + file)
      image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

      if train_val_test < 80:
        image = cv2.resize(image, (300,300))
        X_train.append(image.astype('float16')/255)
        Y_train.append([match.iloc[0]['distance'], match.iloc[0]['angle']])
        if len(X_train) != len(Y_train):
          print("male", file)
          print(len(X_train))
          print(len(Y_train))
      elif train_val_test < 85:
        image = cv2.resize(image, (300,300))
        X_val.append(image.astype('float16')/255)
        Y_val.append([match.iloc[0]['distance'], match.iloc[0]['angle']])
      else:
        image = cv2.resize(image, (300,300))
        X_test.append(image.astype('float16')/255)
        Y_test.append([match.iloc[0]['distance'], match.iloc[0]['angle']])
  else:
    print("Angle " + str(directory) + " skipped")


0


  0%|          | 0/11 [00:00<?, ?it/s]

  0%|          | 0/1100 [00:00<?, ?it/s]

  0%|          | 0/1100 [00:00<?, ?it/s]

  0%|          | 0/1100 [00:00<?, ?it/s]

...

  0%|          | 0/1100 [00:00<?, ?it/s]

  0%|          | 0/1100 [00:00<?, ?it/s]

  0%|          | 0/1100 [00:00<?, ?it/s]

  0%|          | 0/1100 [00:00<?, ?it/s]

  0%|          | 0/1100 [00:00<?, ?it/s]

In [ ]:
gc.collect()

X_train = np.array(X_train)
Y_train = np.array(Y_train, dtype="float32")
print("Shape of Train set's images: " + str(X_train.shape))
print("Shape of Train set's labels: " + str(Y_train.shape))

X_val = np.array(X_val)
Y_val = np.array(Y_val, dtype="float32")
print("Shape of Val set's images: " + str(X_val.shape))
print("Shape of Val set's labels: " + str(Y_val.shape))

X_test = np.array(X_test)
Y_test = np.array(Y_test, dtype="float32")
print("Shape of Test set's images: " + str(X_test.shape))
print("Shape of Test set's labels: " + str(Y_test.shape))

Shape of Train set's images: (9673, 300, 300)
Shape of Train set's labels: (9673, 2)
Shape of Val set's images: (590, 300, 300)
Shape of Val set's labels: (590, 2)
Shape of Test set's images: (1837, 300, 300)
Shape of Test set's labels: (1837, 2)


## Labels Normalization

In [ ]:
mean = np.mean(Y_train[:,0])
std = np.std(Y_train[:,0])
Y_train[:,0] = (Y_train[:,0] - mean)/std
Y_val[:,0] = (Y_val[:,0] - mean)/std
Y_test[:,0] = (Y_test[:,0] - mean)/std

In [ ]:
np.max(Y_train[:,0]), np.min(Y_train[:,0]), np.mean(Y_train[:,0])

(np.float32(1.7337803), np.float32(-1.733804), np.float32(7.256325e-07))

In [ ]:
mean, std

(np.float32(5901007.5), np.float32(633870.7))

## Save dataset

In [ ]:
save_path = "drive/MyDrive/HPPS_Nico/HPPS/Project/ModelRegression/Dataset/TensorflowDataset"

os.makedirs(save_path, exist_ok=True)

np.save(os.path.join(save_path, "X_train.npy"), X_train)
np.save(os.path.join(save_path, "Y_train.npy"), Y_train)
np.save(os.path.join(save_path, "X_val.npy"), X_val)
np.save(os.path.join(save_path, "Y_val.npy"), Y_val)
np.save(os.path.join(save_path, "X_test.npy"), X_test)
np.save(os.path.join(save_path, "Y_test.npy"), Y_test)


## Open Dataset

In [11]:
load_path = "drive/MyDrive/HPPS_Nico/HPPS/Project/ModelRegression/Dataset/TensorflowDataset"

X_train = np.load(os.path.join(load_path, "X_train.npy"))
Y_train = np.load(os.path.join(load_path, "Y_train.npy"))
X_val = np.load(os.path.join(load_path, "X_val.npy"))
Y_val = np.load(os.path.join(load_path, "Y_val.npy"))
X_test = np.load(os.path.join(load_path, "X_test.npy"))
Y_test = np.load(os.path.join(load_path, "Y_test.npy"))


In [12]:
class OpticalDataset(torch.utils.data.Dataset):
    def __init__(self, X, Y):
        self.X = torch.tensor(X, dtype=torch.float32).unsqueeze(1)  # aggiunge il canale
        self.Y = torch.tensor(Y[:,0], dtype=torch.float32)  # solo la distanza normalizzata

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.Y[idx]

train_dataset = OpticalDataset(X_train, Y_train)
val_dataset = OpticalDataset(X_val, Y_val)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)


In [20]:
test_dataset = OpticalDataset(X_test, Y_test)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

## Definition of the models

In [13]:
class StandardModel(nn.Module):
    def __init__(self):
        super(StandardModel, self).__init__()
        self.conv_block = nn.Sequential(
            nn.Conv2d(1, 32, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2),

            nn.Conv2d(32, 64, kernel_size=3),
            nn.ReLU(),
            nn.MaxPool2d(2),

            nn.Conv2d(64, 128, kernel_size=3),
            nn.ReLU(),
            nn.MaxPool2d(2),

            nn.Conv2d(128, 256, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2),

            nn.Conv2d(256, 384, kernel_size=3),
            nn.ReLU(),
            nn.MaxPool2d(2),

            nn.Conv2d(384, 512, kernel_size=3),
            nn.ReLU()
        )
        self.pool = nn.AdaptiveAvgPool2d((1, 1))
        self.regressor = nn.Sequential(
            nn.Flatten(),
            nn.Linear(512, 192),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(192, 1)
        )

    def forward(self, x):
        x = self.conv_block(x)
        x = self.pool(x)
        x = self.regressor(x)
        return x


# Training

In [16]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = StandardModel().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)
criterion = nn.L1Loss()  # Mean Absolute Error

best_val_loss = float('inf')
patience, patience_counter = 10, 0

for epoch in range(1, 1001):
    model.train()
    train_loss = 0
    for inputs, targets in train_loader:
        inputs, targets = inputs.to(device), targets.to(device).unsqueeze(1)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()
        train_loss = loss.item()  # puoi anche accumulare per media

    # Validation
    model.eval()
    val_loss = 0
    with torch.no_grad():
        for inputs, targets in val_loader:
            inputs, targets = inputs.to(device), targets.to(device).unsqueeze(1)
            outputs = model(inputs)
            val_loss += criterion(outputs, targets).item()
    val_loss /= len(val_loader)

    print(f"Epoch {epoch:03d} - Train Loss: {train_loss:.6f} - Val Loss: {val_loss:.6f}", end='')

    if val_loss < best_val_loss:
        best_val_loss = val_loss
        torch.save(model.state_dict(), 'best_model.pth')
        patience_counter = 0
        print("  --> New best model saved!")
    else:
        patience_counter += 1
        print(f"  --> No improvement ({patience_counter}/{patience})")

    if patience_counter >= patience:
        print("Early stopping triggered.")
        break


Epoch 001 - Train Loss: 0.253951 - Val Loss: 0.191077  --> New best model saved!
Epoch 002 - Train Loss: 0.156832 - Val Loss: 0.165493  --> New best model saved!
Epoch 003 - Train Loss: 0.084020 - Val Loss: 0.093905  --> New best model saved!
Epoch 004 - Train Loss: 0.089566 - Val Loss: 0.074658  --> New best model saved!
Epoch 005 - Train Loss: 0.079753 - Val Loss: 0.047771  --> New best model saved!
Epoch 006 - Train Loss: 0.109635 - Val Loss: 0.118295  --> No improvement (1/10)
Epoch 007 - Train Loss: 0.106978 - Val Loss: 0.074043  --> No improvement (2/10)
Epoch 008 - Train Loss: 0.092223 - Val Loss: 0.037553  --> New best model saved!
Epoch 009 - Train Loss: 0.100580 - Val Loss: 0.056200  --> No improvement (1/10)
Epoch 010 - Train Loss: 0.146223 - Val Loss: 0.034641  --> New best model saved!
Epoch 011 - Train Loss: 0.085392 - Val Loss: 0.028198  --> New best model saved!
Epoch 012 - Train Loss: 0.109263 - Val Loss: 0.024603  --> New best model saved!
Epoch 013 - Train Loss: 0.06

## Save the model and collect garbage for resource issues of Colab's RAM

In [17]:
%ls

best_model.pth  drive/  sample_data/


In [18]:
import os

EXPORT_DIR = "drive/MyDrive/HPPS_Nico/HPPS/Project/ModelRegression/Weights/Size300"
MODEL_NAME = "standardRegressionModel_pytorch_datasetTensorflow01_nameMappingInOrder_1100Class_80train5val15test_lr1e-4"

SAVE_DIR = os.path.join(EXPORT_DIR, MODEL_NAME)
os.makedirs(SAVE_DIR, exist_ok=True)

#model.save(os.path.join(SAVE_DIR, f"{MODEL_NAME}.keras"))
torch.save(model.state_dict(), os.path.join(SAVE_DIR, f"{MODEL_NAME}.pth"))
gc.collect()

6471

## Predictions

In [21]:
from torchinfo import summary  # Assicurati che torchinfo sia installato

# Riassunto del modello
summary(model, input_size=(1, 1, 300, 300))

# Dettagli layer
for name, layer in model.named_modules():
    if isinstance(layer, (nn.Conv2d, nn.Linear, nn.MaxPool2d, nn.Dropout)):
        print(f"Layer: {name}")
        if isinstance(layer, nn.Conv2d):
            print(f"  Conv2d -> Filters: {layer.out_channels}, Kernel: {layer.kernel_size}, Stride: {layer.stride}, Padding: {layer.padding}")
        if isinstance(layer, nn.Linear):
            print(f"  Linear -> Units: {layer.out_features}")
        if isinstance(layer, nn.MaxPool2d):
            print(f"  MaxPool2d -> Pool size: {layer.kernel_size}")
        if isinstance(layer, nn.Dropout):
            print(f"  Dropout -> Rate: {layer.p}")
        print()

# Predizioni su test set
model.load_state_dict(torch.load('best_model.pth'))
model.eval()

with torch.no_grad():
    preds = []
    targets = []
    for inputs, target in test_loader:
        inputs = inputs.to(device)
        output = model(inputs).cpu().numpy()
        preds.extend(output)
        targets.extend(target.numpy())

preds = np.array(preds).squeeze()
targets = np.array(targets)

# Rescaling
mean = np.float32(5901007.5)
std = np.float32(633870.7)
y_test_pred_rescaled = preds * std + mean
Y_test_rescaled = targets * std + mean

# Errori
errors = np.abs(Y_test_rescaled - y_test_pred_rescaled) / 1000  # micrometri
avg_error = np.mean(errors)
max_error = np.max(errors)
min_error = np.min(errors)

# Salvataggio su file
output_text = (
    f"Average error: {avg_error} micrometers\n"
    f"Maximum error: {max_error} micrometers\n"
    f"Minimum error: {min_error} micrometers\n"
)
output_file = os.path.join(SAVE_DIR, "error_report.txt")
with open(output_file, "w") as f:
    f.write(output_text)

# Stampa
print(f"Average error is {avg_error:.6f} micrometers")
print(f"Maximum error is {max_error:.3f} micrometers")
print(f"Minimum error is {min_error:.4f} micrometers")


Layer: conv_block.0
  Conv2d -> Filters: 32, Kernel: (3, 3), Stride: (1, 1), Padding: (1, 1)

Layer: conv_block.2
  MaxPool2d -> Pool size: 2

Layer: conv_block.3
  Conv2d -> Filters: 64, Kernel: (3, 3), Stride: (1, 1), Padding: (0, 0)

Layer: conv_block.5
  MaxPool2d -> Pool size: 2

Layer: conv_block.6
  Conv2d -> Filters: 128, Kernel: (3, 3), Stride: (1, 1), Padding: (0, 0)

Layer: conv_block.8
  MaxPool2d -> Pool size: 2

Layer: conv_block.9
  Conv2d -> Filters: 256, Kernel: (3, 3), Stride: (1, 1), Padding: (1, 1)

Layer: conv_block.11
  MaxPool2d -> Pool size: 2

Layer: conv_block.12
  Conv2d -> Filters: 384, Kernel: (3, 3), Stride: (1, 1), Padding: (0, 0)

Layer: conv_block.14
  MaxPool2d -> Pool size: 2

Layer: conv_block.15
  Conv2d -> Filters: 512, Kernel: (3, 3), Stride: (1, 1), Padding: (0, 0)

Layer: regressor.1
  Linear -> Units: 192

Layer: regressor.3
  Dropout -> Rate: 0.3

Layer: regressor.4
  Linear -> Units: 1

Average error is 9.761523 micrometers
Maximum error is 7